# Using Twitter API with Tweepy

To interface with Twitter API, we can use third-party package such as Tweepy.
To use the package, we will need to register and get keys from twitter developer portal.  Then, we use these keys to authenticate with OAuth2 to access twitter API.

In [1]:
import tweepy
import pandas as pd
import pytz
import yaml

In [6]:
with open('twitter.yml') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

In [7]:
auth = tweepy.OAuth2BearerHandler(config['bearer_token'])
api = tweepy.API(auth, wait_on_rate_limit=True)

Tweepy provides many features:
- searching and listing users' information
- reading tweets from user timelines
- creating, fetching, retweeing tweets
- managing followers
- adding and removing likes
- blocking users
- searching and filtering tweets
- listing trends
- streaming tweets in real-time

Let's create a helper function to print a tweet.  The function prints only one line containing, timestamp (in BKK timezone), user who creates tweet, and the content of the tweet.

In [4]:
def print_tweet(tweet):
    # tweet.created_at is an "unaware" timezone, but it acutally is a UTC timezone
    # we will have to make it a UTC first, then convert it to bkk
    utc_dt = tweet.created_at.replace(tzinfo=pytz.UTC)
    timezone_bkk = pytz.timezone('Asia/Bangkok')
    bkk_dt = utc_dt.astimezone(timezone_bkk)
    print('{} [{}] {}'.format(bkk_dt, tweet.user.name, tweet.text[:50]))

## Getting user's information
With get_user method, we can access user's public information such as screen_name, description, followers' count, etc.  Please refer to [Python – User object in Tweepy](https://www.geeksforgeeks.org/python-user-object-in-tweepy/) for more details of user object.

In [9]:
user = api.get_user(screen_name='katyperry')

In [10]:
print('Name:', user.name)
print('Screen Name:', user.screen_name)
print('Description:', user.description)
print('Number of Followers:', user.followers_count)
print('Number of Followings:', user.friends_count)
print('Number of Tweets/Retweets:', user.statuses_count)
print('Headshot\'s URL', user.profile_image_url)

Name: KATY PERRY
Screen Name: katyperry
Description: LOVE is the key that unlocks every door🗝️♥️
Number of Followers: 107168849
Number of Followings: 241
Number of Tweets/Retweets: 11954
Headshot's URL http://pbs.twimg.com/profile_images/1392465354622791687/w_KwtKcE_normal.jpg


## Accessing user timelines
We can get tweets, replies, and mentions in user's timeline, as long as it is public.

In [11]:
timeline = api.user_timeline(user_id=user.id)
for tweet in timeline:
    print_tweet(tweet)

Forbidden: 403 Forbidden
453 - You currently have access to a subset of Twitter API v2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.twitter.com/en/portal/product

## Listing Trends
Trends are location-oriented.  We will have to use location coordinate to get trending information.

In [12]:
# Let's use BKK location
location_info = api.closest_trends(13.739060668870644, 100.53214799610562)

Forbidden: 403 Forbidden
453 - You currently have access to a subset of Twitter API v2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.twitter.com/en/portal/product

In [ ]:
location_info

Get trending keywords and put them in dataframe

In [ ]:
trendings = api.get_place_trends(location_info[0]['woeid'])
df = pd.DataFrame(trendings[0]['trends'])

In [ ]:
df

In [ ]:
# sort trending based on their tweet volume in descending order
trending_df = df.sort_values('tweet_volume', ascending=False)
trending_df

In [ ]:
# get the keyword with the highest tweet volume
top_trending_keyword = trending_df.iloc[3]['name']
print('Top Trending Keyword = ', top_trending_keyword)

In [ ]:
tweets = api.search_tweets(q=top_trending_keyword, count=10)

In [ ]:
for tweet in tweets:
    print_tweet(tweet)

## Understanding cursor
For Twitter API, those information will be returned with some certain limitations e.g. providing only 20 followers.  If you want more complete list, we will have to use Cursor API.  Cursor API is basically a paging mechanism.

In [ ]:
screen_name = 'natawutn'
user = api.get_user(screen_name=screen_name)

In [ ]:
print('Name:', user.name)
print('Screen Name:', user.screen_name)
print('Description:', user.description)
print('Number of Followers:', user.followers_count)
print('Number of Followings:', user.friends_count)
print('Number of Tweets/Retweets:', user.statuses_count)
print('Headshot\'s URL', user.profile_image_url)

In [ ]:
for friend in user.followers():
    print(friend.screen_name)

In [ ]:
# Iterate through all of the account friends
for friend in tweepy.Cursor(api.get_followers, screen_name=screen_name).items():
    # Process the friend here
    print(friend.screen_name)